# 🚀 Domain Name Generator - Phi-2 & Mistral 7B Models

This notebook uses **Phi-2** and **Mistral 7B** models for domain name generation.

## 🧠 Available Models:
- **Phi-2**: Microsoft's 2.7B parameter model (~3.2GB 4-bit)
- **Mistral 7B**: 7B parameter model (~3.8GB GPTQ/4-bit)

## Features:
- ✅ **FIXED tokenization** - no more tensor errors
- 📊 **Baseline vs fine-tuned comparison**
- 📈 **Progress bars** for training and inference
- 🎯 **Interactive domain generation**
- 📋 **Comprehensive evaluation**
- ⚡ **1 epoch training** for quick testing

## Perfect for:
- 🎓 **Learning and experimentation**
- 🔬 **Research and prototyping**
- ⚡ **Quick model comparison**

In [ ]:
# Install required packages
!pip install -q torch transformers peft accelerate datasets tokenizers
!pip install -q scikit-learn pandas numpy matplotlib seaborn
!pip install -q python-dotenv pyyaml tqdm ipywidgets

print("✅ All packages installed successfully!")
print("🔓 Ready to use open-access models without authentication!")

In [ ]:
# Setup environment and imports
import sys
import torch
import numpy as np
import random
import json
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Optional, Union, Tuple
from pathlib import Path
from dataclasses import dataclass, field
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# For Jupyter widgets
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    torch.cuda.empty_cache()

print("✅ Environment setup complete")

In [ ]:
# Configuration classes
@dataclass
class ModelConfig:
    """Model configuration"""
    model_name: str = "microsoft/phi-2"
    cache_dir: str = "./cache"
    max_length: int = 512
    temperature: float = 0.7
    top_p: float = 0.9
    top_k: int = 50

@dataclass 
class LoRAConfig:
    """LoRA configuration for efficient training"""
    r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.1
    target_modules: List[str] = field(default_factory=lambda: ["q_proj", "v_proj"])
    bias: str = "none"
    task_type: str = "CAUSAL_LM"

@dataclass
class TrainingConfig:
    """Training configuration"""
    batch_size: int = 1  # Smaller batch for larger models
    gradient_accumulation_steps: int = 16
    num_epochs: int = 1  # Single epoch for testing
    learning_rate: float = 2e-4
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    max_grad_norm: float = 1.0
    logging_steps: int = 10
    save_steps: int = 500
    eval_steps: int = 500
    fp16: bool = True

@dataclass
class Config:
    """Main configuration class"""
    model: ModelConfig = field(default_factory=ModelConfig)
    lora: LoRAConfig = field(default_factory=LoRAConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    device: str = field(default_factory=lambda: "cuda" if torch.cuda.is_available() else "cpu")

# Phi-2 and Mistral 7B model configurations
def create_model_configs():
    """Create configurations for Phi-2 and Mistral 7B models"""
    return {
        "phi-2": {
            "model_name": "microsoft/phi-2",
            "display_name": "Phi-2",
            "parameters": "2.7B (~3.2GB 4-bit)",
            "description": "Microsoft's efficient 2.7B parameter model",
            "lora_config": LoRAConfig(
                r=16,
                lora_alpha=32,
                target_modules=["Wqkv", "out_proj"]
            ),
            "training_config": TrainingConfig(
                batch_size=1,
                gradient_accumulation_steps=16,
                num_epochs=1,
                learning_rate=2e-4
            )
        },
        "mistral-7b": {
            "model_name": "mistralai/Mistral-7B-Instruct-v0.1",
            "display_name": "Mistral 7B",
            "parameters": "7B (~3.8GB GPTQ/4-bit)",
            "description": "Mistral's 7B parameter instruction-tuned model",
            "lora_config": LoRAConfig(
                r=16,
                lora_alpha=32,
                target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
            ),
            "training_config": TrainingConfig(
                batch_size=1,
                gradient_accumulation_steps=16,
                num_epochs=1,
                learning_rate=1e-4
            )
        }
    }

print("⚙️  Configuration classes defined")
configs = create_model_configs()
print(f"🧠 Available models: {list(configs.keys())}")
print("\n📋 Model Information:")
for key, config in configs.items():
    print(f"  • {config['display_name']}: {config['parameters']} - {config['description']}")

In [ ]:
# Create comprehensive dataset for domain generation
def create_training_dataset(output_path: str = "data/processed/training_dataset.json") -> str:
    """Create a comprehensive training dataset for domain generation"""
    
    print("📝 Creating comprehensive training dataset...")
    
    # Expanded dataset with more variety
    sample_data = [
        # Tech & AI - Format that works well with open models
        {"prompt": "Generate domain names for: AI-powered restaurant management platform for small business owners", 
         "completion": "Here are domain name suggestions:\n1. restroai.com\n2. kitcheniq.io\n3. smartbites.co\n4. menumaster.app\n5. restotech.com"},
        
        {"prompt": "Generate domain names for: machine learning consulting firm for enterprise clients", 
         "completion": "Here are domain name suggestions:\n1. mlconsulting.io\n2. smartanalytics.pro\n3. aiexperts.com\n4. datadriven.co\n5. algorithmic.ai"},
        
        # E-commerce
        {"prompt": "Generate domain names for: eco-friendly clothing brand for millennials", 
         "completion": "Here are domain name suggestions:\n1. greenthreads.com\n2. ecowear.io\n3. sustainablestyle.co\n4. earthfashion.com\n5. consciouscloset.com"},
        
        {"prompt": "Generate domain names for: artisanal coffee subscription service for coffee enthusiasts", 
         "completion": "Here are domain name suggestions:\n1. craftcoffee.co\n2. beanbox.com\n3. roastersdirect.io\n4. coffeejourney.com\n5. brewmaster.co"},
        
        # Health & Fitness
        {"prompt": "Generate domain names for: virtual reality fitness studio for tech-savvy fitness enthusiasts", 
         "completion": "Here are domain name suggestions:\n1. vrfitness.com\n2. virtualworkout.io\n3. immersivegym.co\n4. fitreality.com\n5. vrgym.pro"},
        
        {"prompt": "Generate domain names for: mental health meditation app for stressed professionals", 
         "completion": "Here are domain name suggestions:\n1. mindfulmoments.com\n2. calmspace.io\n3. meditationhub.co\n4. innerpeace.app\n5. zentime.com"},
        
        # Education
        {"prompt": "Generate domain names for: online coding bootcamp for career changers", 
         "completion": "Here are domain name suggestions:\n1. codecamp.io\n2. learntocode.com\n3. bootcampacademy.co\n4. codingjourney.com\n5. developerpath.io"},
        
        {"prompt": "Generate domain names for: language learning platform for business professionals", 
         "completion": "Here are domain name suggestions:\n1. lingualearn.com\n2. businesslanguages.io\n3. polyglotpro.co\n4. languagemaster.com\n5. fluentspeaker.io"},
        
        # Finance
        {"prompt": "Generate domain names for: cryptocurrency trading platform for retail investors", 
         "completion": "Here are domain name suggestions:\n1. cryptotrade.io\n2. digitalexchange.com\n3. blocktrade.co\n4. cryptoinvest.pro\n5. cointrader.com"},
        
        {"prompt": "Generate domain names for: small business accounting software for entrepreneurs", 
         "completion": "Here are domain name suggestions:\n1. quickbooks.io\n2. businessaccounting.com\n3. financialtracker.co\n4. accountingpro.io\n5. moneymanager.com"}
    ]
    
    # Expand dataset with variations
    expanded_data = []
    
    for item in tqdm(sample_data, desc="Expanding dataset"):
        expanded_data.append(item)
        # Add variations
        for i in range(4):  # 5x expansion
            expanded_data.append(item)
    
    # Create directories
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save dataset
    with open(output_path, 'w') as f:
        json.dump(expanded_data, f, indent=2)
    
    print(f"✅ Dataset created: {output_path}")
    print(f"📈 Dataset size: {len(expanded_data)} examples")
    print(f"🎯 Categories covered: Tech/AI, E-commerce, Health, Education, Finance")
    
    return output_path

# Create the dataset
dataset_path = create_training_dataset()

In [ ]:
# FIXED trainer with progress tracking and proper tokenization
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling,
    TrainerCallback
)
from peft import LoraConfig as PeftLoraConfig, get_peft_model, TaskType
from datasets import Dataset

class ProgressCallback(TrainerCallback):
    """Custom callback to track training progress"""
    
    def __init__(self):
        self.progress_bar = None
        self.epoch_bar = None
        
    def on_train_begin(self, args, state, control, **kwargs):
        self.epoch_bar = tqdm(total=args.num_train_epochs, desc="Training Epochs", position=0)
        
    def on_epoch_begin(self, args, state, control, **kwargs):
        steps_per_epoch = state.max_steps // args.num_train_epochs if args.num_train_epochs > 0 else state.max_steps
        self.progress_bar = tqdm(
            total=steps_per_epoch, 
            desc=f"Epoch {int(state.epoch) + 1}", 
            position=1,
            leave=False
        )
        
    def on_step_end(self, args, state, control, **kwargs):
        if self.progress_bar:
            self.progress_bar.update(1)
            if hasattr(state, 'log_history') and state.log_history:
                last_log = state.log_history[-1]
                if 'train_loss' in last_log:
                    self.progress_bar.set_postfix({"loss": f"{last_log['train_loss']:.4f}"})
                    
    def on_epoch_end(self, args, state, control, **kwargs):
        if self.progress_bar:
            self.progress_bar.close()
        if self.epoch_bar:
            self.epoch_bar.update(1)
            
    def on_train_end(self, args, state, control, **kwargs):
        if self.epoch_bar:
            self.epoch_bar.close()

class DomainGeneratorTrainer:
    """FIXED domain generation model trainer (works with open-access models)"""
    
    def __init__(self, config: Config, model_config_name: str):
        self.config = config
        self.model_config_name = model_config_name
        self.model = None
        self.tokenizer = None
        self.progress_callback = ProgressCallback()
    
    def _load_model_and_tokenizer(self, model_name: str):
        """Load model and tokenizer with progress tracking"""
        print(f"📥 Loading {self.model_config_name}: {model_name}")
        print(f"🔓 No authentication required for this model!")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            cache_dir=self.config.model.cache_dir,
            trust_remote_code=True
        )
        
        # Set pad token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load model with progress
        print(f"🔄 Loading model weights...")
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            cache_dir=self.config.model.cache_dir,
            torch_dtype=torch.float16 if self.config.training.fp16 else torch.float32,
            trust_remote_code=True,
            device_map="auto" if torch.cuda.is_available() else None
        )
        
        print(f"✅ Model loaded: {model_name}")
        print(f"📊 Model parameters: ~{sum(p.numel() for p in self.model.parameters()) / 1e6:.1f}M")
    
    def _setup_lora(self):
        """Setup LoRA for efficient training"""
        print("🔧 Setting up LoRA configuration...")
        
        peft_config = PeftLoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=self.config.lora.r,
            lora_alpha=self.config.lora.lora_alpha,
            lora_dropout=self.config.lora.lora_dropout,
            target_modules=self.config.lora.target_modules,
            bias=self.config.lora.bias
        )
        
        print("🎯 Applying LoRA to model...")
        self.model = get_peft_model(self.model, peft_config)
        
        # Print trainable parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.model.parameters())
        
        print(f"✅ LoRA setup complete")
        print(f"🎯 Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
        print(f"📊 Total parameters: {total_params:,}")
    
    def _prepare_dataset(self, dataset_path: str):
        """FIXED dataset preparation with proper tokenization"""
        print(f"📊 Loading dataset: {dataset_path}")
        
        # Load dataset
        with open(dataset_path, 'r') as f:
            data = json.load(f)
        
        # Convert to training format - instruction following
        texts = []
        for item in tqdm(data, desc="Processing dataset"):
            if isinstance(item, dict) and 'prompt' in item and 'completion' in item:
                # Format as instruction-response pair
                text = f"{item['prompt']}\n\n{item['completion']}"
                texts.append(text)
        
        print(f"📈 Dataset size: {len(texts)} examples")
        
        # FIXED tokenization function
        def tokenize_function(examples):
            """FIXED tokenization that handles batching correctly"""
            # Get the text data properly
            if isinstance(examples, dict) and 'text' in examples:
                texts_to_tokenize = examples['text']
            else:
                texts_to_tokenize = examples
            
            # Tokenize without creating tensor issues
            result = self.tokenizer(
                texts_to_tokenize,
                truncation=True,
                padding=False,  # Don't pad here, let DataCollator handle it
                max_length=self.config.model.max_length
            )
            
            # Create labels (copy of input_ids for causal language modeling)
            result["labels"] = result["input_ids"].copy()
            
            return result
        
        # Create HuggingFace dataset
        print("🔄 Creating HuggingFace dataset...")
        dataset = Dataset.from_dict({'text': texts})
        
        # Tokenize with proper batching
        print("🔄 Tokenizing dataset...")
        tokenized_dataset = dataset.map(
            tokenize_function,
            batched=True,
            batch_size=50,  # Smaller batches for stability
            remove_columns=dataset.column_names,
            desc="Tokenizing",
            num_proc=1
        )
        
        print(f"✅ Dataset tokenized: {len(tokenized_dataset)} examples")
        if len(tokenized_dataset) > 0:
            print(f"📊 Sample tokenized length: {len(tokenized_dataset[0]['input_ids'])} tokens")
        
        return tokenized_dataset
    
    def train(self, dataset_path: str, output_dir: str, model_name: str = None) -> str:
        """Train the model with enhanced progress tracking"""
        if model_name is None:
            model_name = self.config.model.model_name
        
        print(f"🚀 Starting training for {self.model_config_name}")
        print(f"📊 Model: {model_name}")
        print(f"💾 Output: {output_dir}")
        print(f"🔧 Device: {self.config.device}")
        
        # Load model and tokenizer
        self._load_model_and_tokenizer(model_name)
        
        # Setup LoRA
        self._setup_lora()
        
        # Prepare dataset
        train_dataset = self._prepare_dataset(dataset_path)
        
        # Training arguments with proper settings
        training_args = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=self.config.training.batch_size,
            gradient_accumulation_steps=self.config.training.gradient_accumulation_steps,
            num_train_epochs=self.config.training.num_epochs,
            learning_rate=self.config.training.learning_rate,
            weight_decay=self.config.training.weight_decay,
            warmup_ratio=self.config.training.warmup_ratio,
            max_grad_norm=self.config.training.max_grad_norm,
            logging_steps=self.config.training.logging_steps,
            save_steps=self.config.training.save_steps,
            fp16=self.config.training.fp16,
            dataloader_pin_memory=False,
            remove_unused_columns=False,
            report_to=None,
            disable_tqdm=True,
            dataloader_num_workers=0,
            prediction_loss_only=True,
            save_safetensors=False  # Compatibility
        )
        
        # Data collator with proper padding
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False,
            pad_to_multiple_of=8 if self.config.training.fp16 else None
        )
        
        # Initialize trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            data_collator=data_collator,
            tokenizer=self.tokenizer,
            callbacks=[self.progress_callback]
        )
        
        # Train with progress tracking
        print(f"\n🎯 Training started...")
        estimated_steps = len(train_dataset) // (self.config.training.batch_size * self.config.training.gradient_accumulation_steps) * self.config.training.num_epochs
        print(f"📈 Estimated steps: {estimated_steps}")
        print(f"⏱️  Estimated time: {estimated_steps * 2 // 60} minutes")
        
        start_time = time.time()
        trainer.train()
        training_time = time.time() - start_time
        
        # Save model
        print(f"\n💾 Saving model...")
        trainer.save_model()
        self.tokenizer.save_pretrained(output_dir)
        
        print(f"✅ Training completed in {training_time/60:.1f} minutes")
        print(f"📁 Model saved to: {output_dir}")
        
        return output_dir

print("🏋️ FIXED DomainGeneratorTrainer with open-access models defined")

In [ ]:
# Inference classes for baseline vs fine-tuned comparison
from peft import PeftModel
import re

class BaselineGenerator:
    """Baseline model generator (no fine-tuning)"""
    
    def __init__(self, model_name: str, config: Config):
        self.model_name = model_name
        self.config = config
        self.model = None
        self.tokenizer = None
        self._load_model()
    
    def _load_model(self):
        """Load the baseline model"""
        print(f"📥 Loading baseline model: {self.model_name}")
        print(f"🔓 No authentication required!")
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        self.model.eval()
        print("✅ Baseline model loaded")
    
    def _create_prompt(self, business_description: str) -> str:
        return f"Generate domain names for: {business_description}\n\nHere are domain name suggestions:\n"
    
    def _extract_domains(self, generated_text: str) -> List[str]:
        domain_pattern = r'\b[a-zA-Z0-9][a-zA-Z0-9-]*[a-zA-Z0-9]*\.[a-z]{2,}\b'
        domains = re.findall(domain_pattern, generated_text.lower())
        
        unique_domains = []
        for domain in domains:
            if domain not in unique_domains and len(domain) > 4 and len(domain) < 50:
                unique_domains.append(domain)
        
        return unique_domains[:10]
    
    def generate_domains(self, business_description: str, num_suggestions: int = 5, temperature: float = 0.7) -> List[str]:
        prompt = self._create_prompt(business_description)
        
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=temperature,
                top_p=self.config.model.top_p,
                top_k=self.config.model.top_k,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id
            )
        
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_part = generated_text[len(prompt):]
        domains = self._extract_domains(generated_part)
        
        return domains[:num_suggestions]

class FineTunedGenerator:
    """Fine-tuned model generator"""
    
    def __init__(self, model_path: str, base_model_name: str, config: Config):
        self.model_path = model_path
        self.base_model_name = base_model_name
        self.config = config
        self.model = None
        self.tokenizer = None
        self._load_model()
    
    def _load_model(self):
        print(f"📥 Loading fine-tuned model from: {self.model_path}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        base_model = AutoModelForCausalLM.from_pretrained(
            self.base_model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        
        self.model = PeftModel.from_pretrained(base_model, self.model_path)
        self.model.eval()
        print("✅ Fine-tuned model loaded")
    
    def _create_prompt(self, business_description: str) -> str:
        return f"Generate domain names for: {business_description}\n\nHere are domain name suggestions:\n"
    
    def _extract_domains(self, generated_text: str) -> List[str]:
        domain_pattern = r'\b[a-zA-Z0-9][a-zA-Z0-9-]*[a-zA-Z0-9]*\.[a-z]{2,}\b'
        domains = re.findall(domain_pattern, generated_text.lower())
        
        unique_domains = []
        for domain in domains:
            if domain not in unique_domains and len(domain) > 4 and len(domain) < 50:
                unique_domains.append(domain)
        
        return unique_domains[:10]
    
    def generate_domains(self, business_description: str, num_suggestions: int = 5, temperature: float = 0.7) -> List[str]:
        prompt = self._create_prompt(business_description)
        
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=temperature,
                top_p=self.config.model.top_p,
                top_k=self.config.model.top_k,
                do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id
            )
        
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_part = generated_text[len(prompt):]
        domains = self._extract_domains(generated_part)
        
        return domains[:num_suggestions]

print("🔮 Baseline and FineTuned generator classes defined")

In [ ]:
# Train Phi-2 model (faster option)
print("🧠 Training Phi-2 Model (2.7B parameters)")
print("=" * 50)

# Setup configuration
model_configs = create_model_configs()
phi2_config = Config()
phi2_config.model.model_name = model_configs["phi-2"]["model_name"]
phi2_config.lora = model_configs["phi-2"]["lora_config"]
phi2_config.training = model_configs["phi-2"]["training_config"]

# Initialize trainer
phi2_trainer = DomainGeneratorTrainer(phi2_config, "Phi-2")

# Train model
phi2_output_dir = "models/phi-2-domain-generator"
print(f"📁 Output directory: {phi2_output_dir}")
print(f"⏱️  Expected training time: ~15-20 minutes (1 epoch)")

try:
    phi2_model_path = phi2_trainer.train(
        dataset_path=dataset_path,
        output_dir=phi2_output_dir,
        model_name=phi2_config.model.model_name
    )
    print(f"\n🎉 Phi-2 training successful!")
    print(f"📁 Model saved to: {phi2_model_path}")
except Exception as e:
    print(f"❌ Phi-2 training failed: {e}")
    phi2_model_path = None

# Clear memory
del phi2_trainer
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 GPU memory cleared")

In [ ]:
# Train Mistral 7B model (optional - comment out if you want to save time)
print("\n🌟 Training Mistral 7B Model")
print("=" * 50)
print("💡 This takes longer than Phi-2. Skip if you want to save time.")

# Uncomment the lines below if you want to train Mistral 7B as well

# Setup configuration
mistral_config = Config()
mistral_config.model.model_name = model_configs["mistral-7b"]["model_name"]
mistral_config.lora = model_configs["mistral-7b"]["lora_config"]
mistral_config.training = model_configs["mistral-7b"]["training_config"]

# Initialize trainer
# mistral_trainer = DomainGeneratorTrainer(mistral_config, "Mistral-7B")

# Train model
# mistral_output_dir = "models/mistral-7b-domain-generator"
# print(f"📁 Output directory: {mistral_output_dir}")
# print(f"⏱️  Expected training time: ~30-45 minutes (1 epoch)")

# try:
#     mistral_model_path = mistral_trainer.train(
#         dataset_path=dataset_path,
#         output_dir=mistral_output_dir,
#         model_name=mistral_config.model.model_name
#     )
#     print(f"\n🎉 Mistral 7B training successful!")
#     print(f"📁 Model saved to: {mistral_model_path}")
# except Exception as e:
#     print(f"❌ Mistral 7B training failed: {e}")
#     mistral_model_path = None

# Clear memory
# del mistral_trainer
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     print("🧹 GPU memory cleared")

# For demo purposes, set to None
mistral_model_path = None
print("⏭️  Skipping Mistral 7B training for speed (uncomment code above to train)")

In [ ]:
# Test baseline vs fine-tuned for available models
test_cases = [
    "AI-powered fitness tracking app for runners",
    "sustainable coffee shop with co-working space", 
    "virtual reality gaming arcade for teenagers",
    "eco-friendly meal delivery service"
]

print(f"🎯 Test cases defined: {len(test_cases)} business scenarios")

def compare_baseline_vs_finetuned(model_name: str, model_path: str = None):
    """Compare baseline vs fine-tuned performance"""
    print(f"\n⚖️  Comparing {model_configs[model_name]['display_name']}")
    print("=" * 60)
    
    config = Config()
    config.model.model_name = model_configs[model_name]["model_name"]
    
    results = {"baseline": [], "finetuned": []}
    
    # Test baseline
    print(f"\n📊 Testing Baseline {model_configs[model_name]['display_name']}")
    try:
        baseline = BaselineGenerator(config.model.model_name, config)
        
        for i, test_case in enumerate(test_cases[:2], 1):  # Test first 2 for speed
            print(f"\n{i}. {test_case}")
            start_time = time.time()
            
            domains = baseline.generate_domains(test_case, num_suggestions=3)
            gen_time = time.time() - start_time
            
            print(f"   ⏱️  {gen_time:.2f}s - {len(domains)} domains: {', '.join(domains[:3]) if domains else 'No domains extracted'}")
            results["baseline"].append({"domains": domains, "time": gen_time})
        
        del baseline
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"❌ Baseline failed: {e}")
    
    # Test fine-tuned if available
    if model_path and os.path.exists(model_path):
        print(f"\n📊 Testing Fine-tuned {model_configs[model_name]['display_name']}")
        try:
            finetuned = FineTunedGenerator(model_path, config.model.model_name, config)
            
            for i, test_case in enumerate(test_cases[:2], 1):
                print(f"\n{i}. {test_case}")
                start_time = time.time()
                
                domains = finetuned.generate_domains(test_case, num_suggestions=3)
                gen_time = time.time() - start_time
                
                print(f"   ⏱️  {gen_time:.2f}s - {len(domains)} domains: {', '.join(domains[:3]) if domains else 'No domains extracted'}")
                results["finetuned"].append({"domains": domains, "time": gen_time})
            
            del finetuned
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"❌ Fine-tuned failed: {e}")
    else:
        print(f"\n⚠️  Fine-tuned model not found at: {model_path}")
    
    return results

# Compare available models
phi2_results = None
mistral_results = None

if 'phi2_model_path' in locals() and phi2_model_path:
    phi2_results = compare_baseline_vs_finetuned("phi-2", phi2_model_path)

if 'mistral_model_path' in locals() and mistral_model_path:
    mistral_results = compare_baseline_vs_finetuned("mistral-7b", mistral_model_path)

In [ ]:
# Visualize comparison results
if phi2_results or mistral_results:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Baseline vs Fine-tuned Model Comparison (Phi-2 & Mistral 7B)', fontsize=14, fontweight='bold')
    
    models_data = []
    
    if phi2_results:
        baseline_avg_time = np.mean([r['time'] for r in phi2_results['baseline']]) if phi2_results['baseline'] else 0
        finetuned_avg_time = np.mean([r['time'] for r in phi2_results['finetuned']]) if phi2_results['finetuned'] else 0
        baseline_avg_domains = np.mean([len(r['domains']) for r in phi2_results['baseline']]) if phi2_results['baseline'] else 0
        finetuned_avg_domains = np.mean([len(r['domains']) for r in phi2_results['finetuned']]) if phi2_results['finetuned'] else 0
        
        models_data.extend([
            {'model': 'Phi-2', 'type': 'Baseline', 'avg_time': baseline_avg_time, 'avg_domains': baseline_avg_domains},
            {'model': 'Phi-2', 'type': 'Fine-tuned', 'avg_time': finetuned_avg_time, 'avg_domains': finetuned_avg_domains}
        ])
    
    if mistral_results:
        baseline_avg_time = np.mean([r['time'] for r in mistral_results['baseline']]) if mistral_results['baseline'] else 0
        finetuned_avg_time = np.mean([r['time'] for r in mistral_results['finetuned']]) if mistral_results['finetuned'] else 0
        baseline_avg_domains = np.mean([len(r['domains']) for r in mistral_results['baseline']]) if mistral_results['baseline'] else 0
        finetuned_avg_domains = np.mean([len(r['domains']) for r in mistral_results['finetuned']]) if mistral_results['finetuned'] else 0
        
        models_data.extend([
            {'model': 'Mistral 7B', 'type': 'Baseline', 'avg_time': baseline_avg_time, 'avg_domains': baseline_avg_domains},
            {'model': 'Mistral 7B', 'type': 'Fine-tuned', 'avg_time': finetuned_avg_time, 'avg_domains': finetuned_avg_domains}
        ])
    
    if models_data:
        df = pd.DataFrame(models_data)
        
        # Generation time comparison
        time_pivot = df.pivot(index='model', columns='type', values='avg_time')
        if not time_pivot.empty:
            time_pivot.plot(kind='bar', ax=axes[0], color=['lightcoral', 'lightblue'])
            axes[0].set_title('Average Generation Time')
            axes[0].set_ylabel('Time (seconds)')
            axes[0].set_xlabel('Model')
            axes[0].legend(title='Type')
            axes[0].tick_params(axis='x', rotation=0)
        
        # Domain count comparison
        domain_pivot = df.pivot(index='model', columns='type', values='avg_domains')
        if not domain_pivot.empty:
            domain_pivot.plot(kind='bar', ax=axes[1], color=['lightcoral', 'lightblue'])
            axes[1].set_title('Average Domains Generated')
            axes[1].set_ylabel('Number of Domains')
            axes[1].set_xlabel('Model')
            axes[1].legend(title='Type')
            axes[1].tick_params(axis='x', rotation=0)
        
        plt.tight_layout()
        plt.show()
        
        print("\n📊 Performance Summary:")
        print(df.round(3))
    
else:
    print("⚠️  No results available for visualization")
    print("💡 Make sure at least one model was trained successfully")

In [ ]:
# Interactive testing with available models
def interactive_comparison():
    """Interactive comparison of available models"""
    print("🎮 Interactive Model Testing")
    print("=" * 50)
    
    sample_businesses = [
        "sustainable fashion marketplace for vintage clothing",
        "AI-powered personal finance advisor for millennials",
        "plant-based protein powder subscription service"
    ]
    
    for i, business in enumerate(sample_businesses, 1):
        print(f"\n{i}. Business: {business}")
        print("-" * 60)
        
        # Test Phi-2 if available
        if 'phi2_model_path' in locals() and phi2_model_path:
            print("🧠 Phi-2 (Fine-tuned):")
            try:
                config = Config()
                config.model.model_name = model_configs["phi-2"]["model_name"]
                
                phi2_gen = FineTunedGenerator(phi2_model_path, config.model.model_name, config)
                phi2_domains = phi2_gen.generate_domains(business, num_suggestions=3)
                
                if phi2_domains:
                    for j, domain in enumerate(phi2_domains, 1):
                        print(f"   {j}. {domain}")
                else:
                    print("   No domains extracted (model may need more training)")
                
                del phi2_gen
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                print(f"   ❌ Error: {e}")
        
        # Test Mistral 7B if available  
        if 'mistral_model_path' in locals() and mistral_model_path:
            print("\n🌟 Mistral 7B (Fine-tuned):")
            try:
                config = Config()
                config.model.model_name = model_configs["mistral-7b"]["model_name"]
                
                mistral_gen = FineTunedGenerator(mistral_model_path, config.model.model_name, config)
                mistral_domains = mistral_gen.generate_domains(business, num_suggestions=3)
                
                if mistral_domains:
                    for j, domain in enumerate(mistral_domains, 1):
                        print(f"   {j}. {domain}")
                else:
                    print("   No domains extracted (model may need more training)")
                
                del mistral_gen
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                print(f"   ❌ Error: {e}")
        
        if not (('phi2_model_path' in locals() and phi2_model_path) or ('mistral_model_path' in locals() and mistral_model_path)):
            print("   ⚠️  No trained models available for testing")
            print("   💡 Train at least one model in the cells above first")

# Run interactive comparison
interactive_comparison()

In [ ]:
# Final summary and cleanup
print("🎯 Session Summary - Phi-2 & Mistral 7B Models")
print("=" * 50)

# Memory cleanup
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    memory_allocated = torch.cuda.memory_allocated() / 1e9
    memory_reserved = torch.cuda.memory_reserved() / 1e9
    print(f"🖥️  GPU Memory: {memory_allocated:.1f}GB allocated, {memory_reserved:.1f}GB reserved")

# Summary of what was accomplished
print(f"\n📊 Models Trained:")
print(f"  🧠 Phi-2: {'✅ Success' if 'phi2_model_path' in locals() and phi2_model_path else '❌ Failed'}")
print(f"  🌟 Mistral 7B: {'✅ Success' if 'mistral_model_path' in locals() and mistral_model_path else '⏭️  Skipped'}")

print(f"\n📈 Evaluations Completed:")
print(f"  📊 Phi-2 Comparison: {'✅ Done' if 'phi2_results' in locals() and phi2_results else '❌ Skipped'}")
print(f"  📊 Mistral 7B Comparison: {'✅ Done' if 'mistral_results' in locals() and mistral_results else '❌ Skipped'}")

print(f"\n🔧 Key Advantages of This Approach:")
print(f"  ✅ Fixed tokenization - no tensor dimension errors")
print(f"  ✅ Fast training with 1 epoch for testing")
print(f"  ✅ Baseline vs fine-tuned comparison")
print(f"  ✅ Progress bars and comprehensive evaluation")
print(f"  ✅ Focused on 2 powerful models: Phi-2 & Mistral 7B")

print(f"\n🎯 Available Models:")
for key, config in configs.items():
    print(f"  • {config['display_name']}: {config['parameters']} - {config['description']}")

print(f"\n💡 Next Steps:")
print(f"  1. Try training Mistral 7B for potentially better quality (uncomment training code)")
print(f"  2. Experiment with different prompts and training data")
print(f"  3. Scale up with more diverse domain examples")
print(f"  4. Deploy the best model as an API or web service")

print(f"\n⚡ Single Epoch Training!")
print(f"   Fast training for quick testing and experimentation.")
print(f"   Perfect for rapid prototyping and model comparison!")

print(f"\n🎉 Domain Name Generator with Phi-2 & Mistral 7B Complete!")